In [ ]:
import pandas as pd
import numpy as np
import time

# Retrieve data

We begin by fetching the reduced data using the Python script `data_transfer.py` from the file `df_merged.parquet`, then importing it into Pandas data frame.

In [ ]:
df = pd.read_parquet('../scripts/df_merged.parquet', engine='pyarrow')

In [ ]:
df.head(2)

In [ ]:
df.shape

Here we extract all unique IDs from our data frame:

In [ ]:
unique_ids = df['objectId'].unique().tolist()
len(unique_ids)

#

To calculate the weight values \(w_i\), we use the formula: `w_i` =\begin{cases}
\frac{1}{{\sigma_i^2}}, & \text{if data is available for day } i \\
0, & \text{otherwise}
\end{cases}

In [ ]:
missing_data = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(missing_data, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(missing_data, 0, 1 / (df['nr_sigflux'] ** 2))

In [ ]:
df[['source','dc_weight']].head(3)

# 

# 

# With distance - factor test 

We group the data by shared ID and create `NumPy` arrays for flux, weighted flux, and the source test(if it's a missing day(data)). We also determine the length of each time series.

In [ ]:
grouped = df.groupby('objectId')

F = grouped['dc_flux'].apply(lambda x: x.values).values
W = grouped['dc_weight'].apply(lambda x: x.values).values
source = grouped['source'].apply(lambda x: x.values).values
lengths = grouped['source'].apply(lambda x: len(x)).values

We define the length of our query,window or chunk, along with the limit factor and the size of each window or chunk.

In [ ]:
m=2
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

The `'no_match_test'` function evaluates whether there are any matches in the provided array, which contains the source test values for a window. If the sum of the array is less than or equal to 1, indicating that all values are missing or only one value is present, the function returns -99. Otherwise, it returns -1 to initialize the window's status as 'no match'.

In [ ]:
def no_match_test(array):
    if array.sum() <= 1 : 
        return -99 ## all are missing, or only one is
    return -1 # initialise as no match  # can be modifieted ? 

"`objects`" list contains a subset of the objects we intend to work with.

"`L_max`" is defined to facilitate partial iteration, serving as a debugging aid by allowing a limit to be set on the number of iterations performed.

We initialize the NumPy arrays with `None` values.

In [ ]:
objects = unique_ids[0:1000]
num_objects = len(objects)
L_max = int(num_objects/2)

print("L_max ", L_max)


R = np.empty(num_objects, dtype=object)
alp = np.empty((num_objects, L_max), dtype=object)
d = np.empty((num_objects, L_max), dtype=object)

In [ ]:
indexes_objects = range(num_objects)
Q = [None] * (L_max)

We initialize R using the '`no_match_test`' function.

In [ ]:
for k in indexes_objects:

    n = lengths[k]
    num_chunks = int(n // 2)-m 

    #print(n,num_chunks)

    chunks = np.array([source[k][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
    result = np.array(list(map(no_match_test, chunks)))
    R[k] = result

### Loop to compute the distance of the subsequence in the time series to its nearest neighbor.

In [ ]:
start_time = time.time()


l= 0
while (l < L_max):
    indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
    
    has_non_empty_list = np.any([value.size > 0 for value in indexes_array])
    if not has_non_empty_list:
        print("break , l = ", l )
        break
        
    for k in range(len(R)): # we can remove the loop for here ! ???
          if np.any(indexes_array[k]):
            f = F[k]
            
            index_no_match = indexes_array[k][0]
            #print(f[index_no_match*2 : index_no_match*2 +chunk_size])
            
            Q[l] = f[index_no_match*2 : index_no_match*2 +chunk_size]
            break
            
    for k in range(len(objects)):
        f = F[k]
        w = W[k]
        n = lengths[k]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  
        #print(n, n_c,len(R[k]))



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])


        s_n = s_1[::2] + s_1[1::2]  
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 
            #alpha[:n-j*2] ==> alpha[:]

        d[k][l] = dd[::2] + dd[1::2]
        
        factor_comparison =  d[k][l] <= factor
                
        R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l # explanation follows below!
        
#         print("indexes_array",indexes_array[k])
#         print("factor_comparison",factor_comparison)
#         print("factor_comparison[indexes_array]",factor_comparison[indexes_array[k]])
#         print("indexes_array[factor_comparison[indexes_array]]",indexes_array[k][factor_comparison[indexes_array[k]]])
#         print("R[k]",R[k])
#         print("indexes_array[factor_comparison[indexes_array]]",R[k][indexes_array[k][factor_comparison[indexes_array[k]]]])
#         print()
#         print()
        
        """for i in indexes_array[k]:
            #print(i)
            if d[k][l][i] <= factor : 
                R[k][i] = l"""
     
    #print("l = ",l)
    
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

Let's break down the expression `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l` step by step:

1. `indexes_array[k]`: This selects the array of indexes corresponding to the k-th element of `indexes_array`.
2. `factor_comparison[indexes_array[k]]`: This applies boolean indexing to `factor_comparison` using the indexes from `indexes_array[k]`. It selects only the elements of `factor_comparison` corresponding to the indexes in `indexes_array[k]`.
3. `indexes_array[k][factor_comparison[indexes_array[k]]]`: This gives the indices where the condition `factor_comparison` is true for the k-th element of `indexes_array`.
4. `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]]`: This uses the indices obtained in the previous step to select elements from the k-th row of `R`.
5. `= l`: Finally, it assigns the value `l` to the selected elements of `R[k]`.


In [ ]:
F[0],d[0][0],R[0],Q

In [ ]:
d

In [ ]:
np.savez('nested_arrays.npz', array1=d, array2=R, array3=Q)

# Load the arrays from the .npz file
data = np.load('nested_arrays.npz')

# # Retrieve the arrays from the loaded data
# d = data['array1']
# R = data['array2']
# Q = data['array3']

#

#

# Distinguishing the two cases!

In [ ]:
m=2
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

In [ ]:
objects = unique_ids[0:1000]
num_objects = len(objects)
L_max = int(num_objects/2)

print("L_max ", L_max)


R_r = np.empty(num_objects, dtype=object)
R_g = np.empty(num_objects, dtype=object)
alp = np.empty((num_objects, L_max), dtype=object)
d = np.empty((num_objects, L_max), dtype=object)

In [ ]:
for k in indexes_objects:

    n = lengths[k]
    num_chunks = int(n // 2)-m 

    #print(n,num_chunks)

    chunks_r = np.array([source[k][i*2 : (i*2+chunk_size):2] for i in range(num_chunks)])
    chunks_g = np.array([source[k][1+i*2 : (i*2+chunk_size):2] for i in range(num_chunks)])
    result_r = np.array(list(map(no_match_test, chunks)))
    result_g = np.array(list(map(no_match_test, chunks)))
    R[k] = result

In [ ]:
start_time = time.time()


l= 0
while (l < L_max):
    indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
    
    has_non_empty_list = np.any([value.size > 0 for value in indexes_array])
    if not has_non_empty_list:
        print("break , l = ", l )
        break
        
    for k in range(len(R)): # we can remove the loop for here ! ???
          if np.any(indexes_array[k]):
            f = F[k]
            
            index_no_match = indexes_array[k][0]
            #print(f[index_no_match*2 : index_no_match*2 +chunk_size])
            
            Q[l] = f[index_no_match*2 : index_no_match*2 +chunk_size]
            break
            
    for k in range(len(objects)):
        f = F[k]
        w = W[k]
        n = lengths[k]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  
        #print(n, n_c,len(R[k]))



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1#[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2#[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = alp[k][l]#np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 
            #alpha[:n-j*2] ==> alpha[:]

        d[k][l] = dd#[::2] + dd[1::2]
        
        factor_comparison =  d[k][l] <= factor
                
        R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l # explanation follows below!
        
#         print("indexes_array",indexes_array[k])
#         print("factor_comparison",factor_comparison)
#         print("factor_comparison[indexes_array]",factor_comparison[indexes_array[k]])
#         print("indexes_array[factor_comparison[indexes_array]]",indexes_array[k][factor_comparison[indexes_array[k]]])
#         print("R[k]",R[k])
#         print("indexes_array[factor_comparison[indexes_array]]",R[k][indexes_array[k][factor_comparison[indexes_array[k]]]])
#         print()
#         print()
        
        """for i in indexes_array[k]:
            #print(i)
            if d[k][l][i] <= factor : 
                R[k][i] = l"""
     
    #print("l = ",l)
    
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

#

#

# basic code 

In [ ]:
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids}
#print(d)

count = 0

for k in unique_ids:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

#

#

#

# with l>0

In [ ]:
m=2
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))


this part takes a lot of time ! 

In [ ]:
chunk_size = 2 * (m + 1)
# Calculate the number of chunks
num_chunks = len(df[['dc_flux']]) // chunk_size
# Split the DataFrame into chunks and store them in an array
Q = [df[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]


In [ ]:
if m>0: #for j in range(1, max(1, m+1)): this isn't faster
    for j in range(1, m+1):
        Q.extend([df[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])

Q = pd.concat(Q) # here we transform Q from list of dataframes to a single data frame  
# we will be able to distinct the values by using l variable s


In [ ]:
Number_l = 1#(len(Q)) // chunk_size 

d   = {k: [0] * Number_l for k in unique_ids[0:1]}
alp = {k: [0] * Number_l for k in unique_ids[0:1]}
a = {k: [0] * Number_l for k in unique_ids[0:1]}

In [ ]:
start_time = time.time()

count = 0

for k in unique_ids[0:1]:
    
    T = df[df['objectId'] == k][['dc_flux', 'dc_weight']]
    n = len(T)
    f = T['dc_flux'].values
    w = T['dc_weight'].values
    
    zeros = np.zeros(n, dtype=float)

    for l in range(Number_l): # I think we will changee it to while 
        #print(l)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        d[k][l]=0
        
        for j in range(0,m+1): 
            h = np.tile(Q[2*l*(m+1) +j*2: 2*l*(m+1) + j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
        #alp[k][l] = s
        
        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[2*l*(m+1) + j*2: 2*l*(m+1) +j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            # 2*l*(m+1) this part is to take from Q by l 
            print(Q[2*l*(m+1) + j*2: 2*l*(m+1) +j*2+2])
            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] 

        d[k][l] = dd[::2] + dd[1::2]
        
        a[k][l] =  d[k][l] <= factor   
            
        if (d[k][l] <= factor).all():
            print(k, ": ", d[k][l],"\n")
            count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
Q

In [ ]:
Number_l, len(Q), len(df)

In [ ]:
len(d[k][0])

 j'ai complété l'implémentation `(partie with l>0)`, mais pour 'Q', j'ai actuellement toutes les valeurs possibles pour 'Q' (c'est-à-dire toutes les combinaisons possibles de valeurs de flux)
 dans ce cas le code fait tous les calculs des distances en stockant les valeurs True ou False si d<= factor (variable 'a') . 
 
Cependant, cela rend le calcul trop complexe(avec les donnees completes il ne marche pas (kernel interrepted )). Donc j'essaie de passer à l'approche sélective, mais je rencontre des difficultés avec la logique de sélection, surtout lorsque 'm' est supérieur à zéro.

 ma compréhension du logique de sélection :

- Lorsque 'm' = 0, si la valeur de 'd(i)'  > facteur, je considère 'f(i)' comme la nouvelle valeur de 'Q' (en faites, je prend tous les valeurs f(i) ou d(i)> facteur ce qui réduit 'Q' à ces valeurs)
    
- Mais lorsque 'm' >0 alors que 'Q' contient deux valeurs ou plus (supposons 'i' et 'i+1'), la condition est : si à la fois 'd(i)' et 'd(i+1)' sont supérieurs au facteur, alors 'Q_next' est composé de 'f(i)' et 'f(i+1)' n'est ce pas ?
    
    

#### Mr.  : 
m étant donné. (possiblement m>0)
initialiser R(i) à « no match »
Pour tout i:
	Si Sum (missing(i) .. missing (i+m) ) <=1    # Tout est missing, ou tout sauf 1 est missing
		R_i :=  « missing »
Pour tout l < l_max :    # l_max permet de ne fait qu'une partie de l'itération, utile pour débugguer
	Initialiser Q(l) :
		Soit un i(l) / R(i(l)) = « no match »    # / pour "tel que" 
			si pas possible : fin
		Q(l) := f(i(l)) … f(i(l)+m)
	Pour tout i / R(i) = « no match »  :    
		calculer d(i)
		si d(i) < Limit(m):
			R(i) := l 
Valeurs de retour : Q(l), R(i), et aussi d(i) à titre d'information.

#

#

#

#

# TEST

In [ ]:
list1 = [0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9]
list2 = [[0,0,1,1],[1,1,2,2],[2,2,3,3],[3,3,4,4],[4,4,5,5]]

In [ ]:
m=1
idx =3
chunk_size = 2 * (m + 1)
list1[idx*2:idx*2+chunk_size:2]

In [ ]:
m=1
idx =3
chunk_size = 2 * (m + 1)
list1[idx*2:idx*2+chunk_size]

In [ ]:
R = np.empty(len(objects), dtype=object)

m=1
chunk_size =2 * (m + 1)

for k in range(len(objects)):

    n = len(source[k])
    #num_chunks = n // chunk_size

    #chunks = [source[k][i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]
    num_chunks = int(n // 2)-m 

    print(n,num_chunks)


    chunks = np.array([source[k][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
#     if m>0: 
#         for j in range(1, m+1):
#             chunks.extend([source[k][i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
#     if m > 0:
#         for j in range(1, m + 1):
#             for i in range(num_chunks - 1):
#                 chunks.append(source[k][i * chunk_size + j * 2: (i + 1) * chunk_size + j * 2])

    #result = np.concatenate(np.array([no_match_test(chunk) for chunk in chunks]))  # Apply no_match_test directly
    result = np.array(list(map(no_match_test, chunks)))
    R[k] = result

    ### le

In [ ]:
m=0
for j in range(1, max(1, m+1)):
    print(j*2)

if m>0: 
    for j in range(1, m+1):
        print(j*2)

In [ ]:
import timeit

m = 100

code_snippet_1 = """
s_1 = np.zeros(n, dtype=float)
s_2 = np.zeros(n, dtype=float)
s_3 = np.zeros(n, dtype=float)
s_4 = np.zeros(n, dtype=float)
s_5 = np.zeros(n, dtype=float)
s_6 = np.zeros(n, dtype=float)
"""

code_snippet_2 = """
zeros = np.zeros(n, dtype=float)

s_1 = zeros.copy()
s_2 = zeros.copy()
s_3 = zeros.copy()
s_4 = zeros.copy()
s_5 = zeros.copy()
s_6 = zeros.copy()

"""

time_taken_1 = timeit.timeit(stmt=code_snippet_1, number=10000000, globals=globals())
time_taken_2 = timeit.timeit(stmt=code_snippet_2, number=10000000, globals=globals())

print("Time snippet 1:", time_taken_1)
print("Time snippet 2:", time_taken_2)



#

#

#

# Test for Q all possible values ! 

In [ ]:
import pandas as pd

values = list(range(12))  

# Duplicate each value
duplicated_values = [val for val in values for _ in range(2)]


df2 = pd.DataFrame({'dc_flux': duplicated_values})

#print(df2)
print(duplicated_values)

In [ ]:
f = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11]

In [ ]:
for j in range(0,m+1):
            h = np.tile(Q[0][j*2: j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            print(h)

In [ ]:
# Define the size of each chunk
m=1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = int(len(duplicated_values) // 2) -m 
print(num_chunks,chunk_size)

values =duplicated_values
Q = [duplicated_values[i*2 : (i*2+chunk_size)] for i in range(num_chunks)]
Q,len(Q)


In [ ]:
result = np.array(list(map(no_match_test, np.array(Q))))
result

In [ ]:
Q =[]
# Q =[]
# for i in range(0,num_chunks):
#     Q.append(values[i*2 : (i*2+chunk_size)])
# Q
# for i in range(0,num_chunks):
#     print(i*2)
#     print(values[i*2 : (i*2+chunk_size)])
#     Q.append(values[i : (i*2+chunk_size)])
# Q

In [ ]:
m = 1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = len(duplicated_values) - chunk_size + 1
print(num_chunks)
# Split the DataFrame into overlapping chunks and store them in an array
Q = [values[i : i + chunk_size] for i in range(num_chunks)]
Q

In [ ]:
for j in range(1, m+1):
    Q.extend([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
    #print([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
Q



In [ ]:
O = pd.concat(Q)

# Display the resulting DataFrame
print(O)

In [ ]:
m=2
j=2
l=0
#np.tile(Q[(j*2):(j*2+2)], (4 // 2, 1)).ravel()
O[2*l*(m+1) +(j*2) : 2*l*(m+1) +(j*2+2)]

#

#

#

# distance dict to dataframe 

In [ ]:
d

In [ ]:
list_of_dicts = []

# Iterate through the data dictionary and create dictionaries for each object ID
for object_id, distances in d.items():
    for distance in distances:
        list_of_dicts.append({'objectId': object_id, 'distance': distance})

# Create DataFrame from the list of dictionaries
pdf = pd.DataFrame(list_of_dicts)

# Display the DataFrame
print(pdf)

In [ ]:
len(df)/2898

In [ ]:
pdf['is_distance_gt_factor'] = pdf['distance'] <= factor

In [ ]:
pdf

# Trash

In [ ]:
df[df['objectId'] =='ZTF18acevrat' ]

In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [ ]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d0 = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])
                s = np.where(s_d == 0, 0, s_n / s_d)

                alpha += s
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d0[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d0[k] <= factor).all():
        print(d0[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       